In [70]:
from pyblake2 import blake2b
import hashlib as hl
import hmac
from tezio.Utils import Utils
import pysodium

class BIP32HDKeys():
    
    def __init__(self):
        # do something
        self.secret_phrase = None
        self.secret_seed = None
        self.derivation_path = None
        self.master_sk = None
        self.master_cc = None
        self.child_sk = None
        self.child_cc = None
        self.public_point = None
        self.secret_exponent = None
        self.tzaddress = None
        self.sk_b58 = None
        
        return
    
    def __hmac_sha512(self, key, data):
        I = (hmac.new(key, data, hl.sha512)).digest()
        IL = I[:32]
        IR = I[32:]
        return IL, IR
    
    def __parse_path(self, path: str): # default tezos path
        if path.endswith('/'):
            path = path[:-1]
        path = path.split('/')
        
        if path[0] == 'm' or path[0] == '':
            path = path[1:]
        
        indeces = []
        for each in path:
            if each.endswith("'"): # hardened
                index = (1 << 31) + int(each[:-1])
                index = index.to_bytes(4,'big') # from int to bytes
            else:
                index = int(each)
            indeces.append(index)
    
        return indeces
    
    def __parent_to_child(self, parent_sk, parent_cc, index):
        data = b'\x00' + parent_sk + index
        child_sk, child_cc = self.__hmac_sha512(parent_cc, data)
        return child_sk, child_cc
    
    
    # do double sha256 checksum and return the last four bytes of the digest
    def __sha256_checksum(self, _bytes):
        digest = hl.sha256(hl.sha256(_bytes).digest()).digest()
        return digest[:4]
    
    # 24 word mnemonic phrase to secret seed
    def phrase_to_seed(self, phrase:str, password:str = ''):
        self.secret_phrase = phrase 
        passphrase = 'mnemonic' + password
        self.secret_seed = hl.pbkdf2_hmac('sha512', self.secret_phrase.encode('utf-8'), passphrase.encode('utf-8'), 2048)
        return self.secret_seed.hex()
    
    # calcuolate master secret key and master chaincode from secret seed
    def seed_to_master(self):
        self.master_sk, self.master_cc = self.__hmac_sha512(b'ed25519 seed', self.secret_seed)
        return self.master_sk.hex(), self.master_cc.hex()
    
    def derivation_path_to_keys(self, path:str = "m/44'/1729'/0'/0'"):
        self.derivation_path = path
        indeces = self.__parse_path(self.derivation_path)
        self.child_sk = self.master_sk
        self.child_cc = self.master_cc
        for index in indeces:
            self.child_sk, self.child_cc = self.__parent_to_child(self.child_sk, self.child_cc, index)
    
        return self.child_sk.hex(), self.child_cc.hex()
    
    def base58_checksum(self, data: bytes, prefix: bytes):
        _bytes = prefix + data
        checksum = self.__sha256_checksum(_bytes)
        b58 = Utils.base58(_bytes + checksum)
        return b58
    
    def sk_to_public_point(self, sk): # for Ed25519, the public point is the public key and the secret exponent is the sk || pk
        self.public_point, self.secret_exponent = pysodium.crypto_sign_seed_keypair(seed=sk)
        return self.public_point, self.secret_exponent
        
    def pk_hash(self, prefix: bytes = b'\x06\xa1\x9f'): # default prefix is tz1
        pkh = blake2b(data=self.public_point, digest_size=20).digest()
        self.tzaddress = self.base58_checksum(pkh, prefix)
        return self.tzaddress
    
    def sk_base58(self, prefix: bytes = b'+\xf6N\x07'):
        self.sk_b58 = self.base58_checksum(self.secret_exponent, prefix)
        return self.sk_b58
    
    def pk_base58(self, prefix: bytes = b'\r\x0f%\xd9'):
        self.pk_b58 = self.base58_checksum(self.public_point, prefix)
        return self.pk_b58


   

In [71]:
phrase = 'motor jazz team food when coach reward hidden obtain faculty tornado crew toast inhale purchase conduct cube omit illness carbon ripple thank crew material'

In [72]:
myWallet = BIP32HDKeys()
myWallet.phrase_to_seed(phrase)
myWallet.seed_to_master()
myWallet.derivation_path_to_keys("m/44'/1729'/0'/0'")

('8dbdbca8b45da2953a1a487aa89fda6addda80e006bc6ccf876ff22814ab7f48',
 '71fa2cd0c6d5daf400f4615846f7bd020ef3de0798e14eaa777d2870fd16788e')

In [73]:
myWallet.sk_to_public_point(myWallet.child_sk)

(b'\x06\xc4Z\xa4\x19q\xc6\xd2E\x93g\xde\xfc\x0e\xf7`x\xbc\x02@\xb5dq\xaa\xb40Z\x8f&\xe6t\x84',
 b'\x8d\xbd\xbc\xa8\xb4]\xa2\x95:\x1aHz\xa8\x9f\xdaj\xdd\xda\x80\xe0\x06\xbcl\xcf\x87o\xf2(\x14\xab\x7fH\x06\xc4Z\xa4\x19q\xc6\xd2E\x93g\xde\xfc\x0e\xf7`x\xbc\x02@\xb5dq\xaa\xb40Z\x8f&\xe6t\x84')

In [74]:
myWallet.pk_hash(b'\x06\xa1\x9f') # tz1 prefix


'tz1MN2TrBeSeTvj7Lu3RxeqRwvTcBBo98iza'

In [75]:
myWallet.sk_base58(b'+\xf6N\x07') # edsk prefix

'edskRvZKHaBdkQH8tkJZRVUF6kkSdXpEdBhJGFJh2fj9EgyEsAj2fTFNCUrBb2R33rwFgV9MLaqftUsSMqR5PqCL6omkmBMwBC'

In [76]:
myWallet.pk_base58(b'\r\x0f%\xd9') # 

'edpkthCnqGYzzKZa3YFJYDDafa5xnZxdQQsHXh8n7qwFu1JRpRAE94'

In [2]:
# SLIP10 - alternative to BIP32


seed = mnemonic_to_seed(phrase) # mnemonic to seed
master_sk, master_cc = seed_to_master(seed) # seed to master secret key and chaincode 
child_sk, child_cc = derivation_path_to_keys(master_sk, master_cc, "m/44'/1729'/0'/0'") # master sk and cc to child sk and cc
public_point, secret_exponent = sk_to_public_point(child_sk) # public point and secret exponent
tzaddress = pk_hash(public_point, prefix = b'\x06\xa1\x9f') # prefix b'tz1'
secret = blake2b_checksum(secret_exponent, prefix = b'+\xf6N\x07') # prefix b'edsk'
pk = blake2b_checksum(public_point, prefix = b'\r\x0f%\xd9')
print(tzaddress)
print(secret)
print(pk)

NameError: name 'mnemonic_to_seed' is not defined

In [48]:
def sha256_checksum(_bytes):
    digest = hl.sha256(hl.sha256(_bytes).digest()).digest()
    return digest[:4]

In [49]:
myWallet.master_sk

b'\xdb\xfa\xc6G\x903\x96\xf2JeiLU!\x86SgN0A\xb6e\xe0\x8a\xfc\xb4S\x1f3@\xf7j'

In [51]:
sha256_checksum(myWallet.master_sk).hex()

'5dcc97cd'